In [14]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import datetime as dt
import json
import time

In [16]:
df = pd.DataFrame(columns=['brand', 'model', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023'])

In [17]:
brands = ["audi", "bmw", "citroen", "dacia", "fiat", "ford", "honda", "hyundai", "jeep", "kia", "mercedes", "mitsubishi", "nissan", "opel", "peugeot", "renault", "skoda", "suzuki", "toyota", "volkswagen", "volvo"]

In [18]:
def models(brand):
    url = f"http://www.arabalar.com.tr/{brand}"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    time.sleep(1)
    models_soup = soup.find_all('a', {'class': 'list-group-item'})
    models_soup = list(models_soup)
    
    models_list = []    
    for model in models_soup:
        models_list.append(model.text.lower().replace(' ', '-'))

    return models_list
    


In [19]:
for brand in brands:
    for model in models(brand):
        df = df.append({'brand': brand, 'model':model}, ignore_index=True) # type: ignore

In [20]:
df.to_pickle('car_dataset.pkl')

In [21]:
df

,brand,model,2016,2017,2018,2019,2020,2021,2022,2023
0,audi,a3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,audi,a4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,audi,q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,audi,q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,audi,q5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
422,volvo,c70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423,volvo,s40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
424,volvo,s70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425,volvo,v50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df_brand_with_model = df.copy()

In [23]:
def insert_model_prices(brand, model):
    
    counter = 0

    for year in range(2016,2023):      
        
        response_from_wm = requests.get(f"http://archive.org/wayback/available?url=http://www.arabalar.com.tr/{brand}/{model}/{year}&timestamp={year}")
        print(response_from_wm)
        counter += 1

        time.sleep(np.random.randint(2, 5))

        try:
            response_from_wm = response_from_wm.json()
        except ValueError:
            print("JSON DECODE ERROR BUT CONTINUING...")
            continue 
        
        if response_from_wm['archived_snapshots'] == {}:
            continue
        

        wm_url = response_from_wm['archived_snapshots']['closest']['url']
        # headers = {'User-Agent': 'Mozilla/5.0'}
        # print("URL: " + wm_url + "\nCONTENT: \n")
        
        response = requests.get(wm_url)
        soup = BeautifulSoup(response.content, 'lxml')

        prices_html = soup.find_all('li', {'class': 'list-group-item'})
        prices_element = list(prices_html[0])
        prices_element = str(prices_element)

        # print(prices_element)
        start_index = prices_element.index('span') + 5
        # print(prices_element[start_index:])
        stop_index  = prices_element.rindex('<a') - 4
        # print(prices_element[:stop_index])
        try:
            price = int(prices_element[start_index:stop_index].replace('.', ''))
        except:
            price = np.nan    
        # print("price: " + str(price) + " TYPE: " + str(type(price)))

        df_brand_with_model.loc[(df['brand'] == brand) & (df['model'] == model), str(year)] = price

        
        

        if counter % 100 == 0:
            #df_brand_with_model.to_pickle('car_dataset_with_prices.pkl')

            #delete first 10 rows of df_brand_with_model
            #df_brand_with_model = df_brand_with_model.iloc[10:]

            #print("SAVED TO PICKLE")
            time.sleep(10)

In [24]:
for brand in df['brand'].unique():
    for model in df[df['brand'] == brand]['model']: 
        insert_model_prices(brand, model)

<Response [429]>
JSON DECODE ERROR BUT CONTINUING...
<Response [429]>
JSON DECODE ERROR BUT CONTINUING...
<Response [429]>
JSON DECODE ERROR BUT CONTINUING...
<Response [429]>
JSON DECODE ERROR BUT CONTINUING...
<Response [429]>
JSON DECODE ERROR BUT CONTINUING...
<Response [429]>
JSON DECODE ERROR BUT CONTINUING...
<Response [429]>
JSON DECODE ERROR BUT CONTINUING...
<Response [429]>
JSON DECODE ERROR BUT CONTINUING...
<Response [429]>
JSON DECODE ERROR BUT CONTINUING...
<Response [429]>
JSON DECODE ERROR BUT CONTINUING...
<Response [429]>
JSON DECODE ERROR BUT CONTINUING...
<Response [429]>
JSON DECODE ERROR BUT CONTINUING...
<Response [429]>
JSON DECODE ERROR BUT CONTINUING...
<Response [429]>
JSON DECODE ERROR BUT CONTINUING...
<Response [429]>
JSON DECODE ERROR BUT CONTINUING...
<Response [429]>
JSON DECODE ERROR BUT CONTINUING...
<Response [429]>
JSON DECODE ERROR BUT CONTINUING...
<Response [429]>
JSON DECODE ERROR BUT CONTINUING...
<Response [429]>
JSON DECODE ERROR BUT CONTINU

KeyboardInterrupt: 

In [ ]:
df

,brand,model,2016,2017,2018,2019,2020,2021,2022,2023
0,audi,a3,136650,192870,256210,250830,300710,707130,971670,NaN
1,audi,a4,179540,259080,350942,403520,740350,1217000,1588300,NaN
2,audi,q2,142340,168220,248840,270880,408180,765100,1028000,NaN
3,audi,q3,172870,198180,331490,NaN,NaN,966500,1274000,NaN
4,audi,q5,250950,439140,671470,591310,631620,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
422,volvo,c70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423,volvo,s40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
424,volvo,s70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425,volvo,v50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# def snapshots(brand_url):

#     counter_timestamp = 2023
#     old_timestamp = 0

#     while True:
#         response = requests.get(f"http://archive.org/wayback/available?url={brand_url}&timestamp={counter_timestamp}")
#         response = response.json()
#         new_timestamp = response['archived_snapshots']['closest']['timestamp'] 
        
#         if old_timestamp == new_timestamp:
#             break
#         else: 
#             old_timestamp = new_timestamp
#             counter_timestamp -= 1

#     return counter_timestamp+2
        

In [ ]:
# for url in df['brand_url']:
#     df.loc[df['brand_url'] == url, 'oldest_snapshot'] = snapshots(url)

In [ ]:
# df

In [ ]:
#i dont want that cell to be give output
